In [1]:
#cd ..

/home/rodrigue/dev/web_scraping


In [ ]:
from isy_timetable.timetable_scraper import TimetableScraper,semesters,timetable_types_value
from requests_html import HTML
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
scraper = TimetableScraper(group="АС-112Б",semester=2,academic_year='2022/2023', headless=True)
driver = scraper.get_driver()
driver.get(scraper.endpoint)
page = driver.page_source
assert "Расписание" in page

In [ ]:
semester_select = driver.find_element(By.NAME,"schedule_semestr_id")
semester = semesters[scraper.semester].format(academic_year=scraper.academic_year)
for term in semester_select.find_elements(By.TAG_NAME,"option"):
    if term.text == semester:
        term.click()
        break


In [ ]:
filter_radio = driver.find_element(By.CSS_SELECTOR,"input[value='1']")
filter_radio.click()

In [ ]:
form = driver.find_element(By.TAG_NAME,"form")
form.submit()

In [ ]:
group_select = driver.find_element(By.CSS_SELECTOR,"select[name='student_group_id']")
for group in group_select.find_elements(By.TAG_NAME,"option"):
    if scraper.group in group.text:
        group.click()
        break

In [ ]:
form = driver.find_element(By.TAG_NAME,"form")
form.submit()

In [ ]:
src_text = driver.page_source
html_obj = HTML(html=src_text)

In [ ]:
timetable_table = html_obj.find("table", first=True)
headers = []
table_header = timetable_table.find("thead", first=True)
for td in table_header.find("td"):
    headers.append(td.text)

timetables = {}
table_body = timetable_table.find("tbody", first=True)
day = None
for tr in table_body.find("tr"):
    tds = tr.find("td")
    if len(tds) == 1:
        day = tds[0].text
        timetables[day] = []    
    else:
        timetable = {}
        for header,td in zip(headers[1:],tds):
            timetable[header] = td.text
        timetables[day].append(timetable)


timetables

In [ ]:
# Test scraper for group
scraper1 = TimetableScraper(group="АС-112Б",semester=2,academic_year='2022/2023', headless=True)
scraper1.get_timetables_dict()

In [ ]:
scraper2 = TimetableScraper(teacher="Абдрахманова",semester=2,academic_year='2022/2023', headless=True)
scraper2.select_semester()
filter_radio = scraper2.driver.find_element(By.CSS_SELECTOR,"input[value='2']")
filter_radio.click()
scraper2._submit_form()
teacher_select = scraper2.driver.find_element(By.CSS_SELECTOR, "select[name='teacher']")
for teacher in teacher_select.find_elements(By.TAG_NAME, "option"):
    if scraper2.teacher.lower() in teacher.text.lower():
        teacher.click()
        break
else:
    raise ValueError(f"teacher {scraper2.teacher} not found.")
scraper2._submit_form()
scraper2.get_timetables_dict()

In [ ]:
# Test scraper for teacher
scraper3 = TimetableScraper(teacher="Абдрахманова",semester=2,academic_year='2022/2023', headless=True)
scraper3.get_timetables_dict()